In [13]:
import numpy as np
import pandas as pd
from functools import partial


In [4]:
from ISLP import load_data
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, accuracy_score


In [5]:
np.random.seed(1)

In [6]:
default_df = load_data("Default")
default_df["default_res" ] = np.where(default_df["default"] == "Yes", 1, 0)
default_df


,default,student,balance,income,default_res
0,No,No,729.526495,44361.625074,0
1,No,Yes,817.180407,12106.134700,0
2,No,No,1073.549164,31767.138947,0
3,No,No,529.250605,35704.493935,0
4,No,No,785.655883,38463.495879,0
...,...,...,...,...,...
9995,No,No,711.555020,52992.378914,0
9996,No,No,757.962918,19660.721768,0
9997,No,No,845.411989,58636.156984,0
9998,No,No,1569.009053,36669.112365,0



## a)

In [35]:
X = default_df[['income', 'balance']]
y = default_df['default_res']

In [36]:
model = sm.GLM(y, X, family=sm.families.Binomial())

result = model.fit()
result.summary()
result.bse

income     0.000004
balance    0.000070
dtype: float64

## b)

In [37]:
def boot_fn(
        X,
        y,
        data,
        idx,
):
    filt_data = data.loc[idx]
    predictors = filt_data[X]
    response = filt_data[y]
    model = sm.GLM(response, predictors, family=sm.families.Binomial())

    result = model.fit()

    coefficient_0 = result.params[0]
    coefficient_1 = result.params[1]

    return result.params


In [24]:
partial_OLS = partial(
        boot_fn,
        ['income', 'balance'],
        ['default_res'],
    
)

In [23]:
 def boot_SE(
    func,
    D,
    n=None,
    B=100,
    seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
        n,
        replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [32]:
hp_se = boot_SE(
    partial_OLS,
    default_df,
)
hp_se

income     0.000004
balance    0.000070
dtype: float64

In [31]:
result.bse

income     0.000004
balance    0.000070
dtype: float64